<a href="https://colab.research.google.com/github/Vipulabhiram/GenAI_Assignment_1/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers faiss-cpu pymupdf sentence-transformers
#Sentence transformer hep to automaically embed by using modal no need to write manual code for embedding
#pymupdf  is a library used to import fitz which is rquired to operate with pdf like to open pdf or to rortate pages

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12

Inguestion

In [ ]:
#As we will be getting the document from my files
#upload function is helped choose the pdf to get access from our files and t will download our pdf maually
from google.colab import files

#down first lines of code used to pickup files and secound line of code is used for
uploaded = files.upload()
pdf_path=list(uploaded.keys())[0]

Saving Top 50 LLM Interview Questions.pdf to Top 50 LLM Interview Questions.pdf


In [ ]:
#Here we will import fitz which will help us to work with pdfs and open function will be helped to open the pdf
#In this cell of code we will be getting the content of pdf into a single string which we had named a text
import fitz
def extract_text_from_pdf(pdf_path):
  doc = fitz.open(pdf_path)
  text=""
  for page in doc:
    text+=page.get_text()
  return text

document_text = extract_text_from_pdf(pdf_path)


Chunking

In [ ]:
#Here we are import textwrap which is used to make the chunks readable
#Here in this stage we will be breeaking the content into small small parts(chunks)
import textwrap
def chunk_text(text,chunk_size=300):
  return textwrap.wrap(text,width=chunk_size)

chunks = chunk_text(document_text)

print("sample_chunk:",chunks[0])

sample_chunk: TOP TOP TOP 50 LLM 50 LLM 50 LLM Bhavishya Pandit Interview Questions Interview Questions Q1. What is tokenization, and why is it important in LLMs? Ans - Tokenization is the process of splitting text into smaller units called tokens, which can be words, subwords, or even characters. For instance,


Embedding, Indexing

In [ ]:
#SentenceTransformer is been imported from sentence_transformers library for embedding purpose by using embedding modal
#here we are using .encode function for embedding it will convert our chunks into vectors
#here in index we will storing a single dimention vector and we are connverting them into array by using numpy library.
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

#embedding part
embed_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embed_model.encode(chunks)

#indexing part
dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))


Retrieve, Generation

In [ ]:
#here we are importing pipeline function from transformers library to make generate the output for assigning a nlp task to a particular model
#here we are getting the query and embedding it after that getting the relavent vector from index by using search function
#by passing the query that we had embedded and converting it into natural language and as it will be present in list formate joining all together
#by join function
#creating a prompt by a context and query of user and passing that prompt to the pipeline so that it will be generate the answer and return the output.
from transformers import pipeline

generation_pipeline = pipeline("text2text-generation",model="google/flan-t5-small")

def retrieve_and_generate(query,top_k=1):
  query_embedding = embed_model.encode([query])
  _,indices = index.search(np.array(query_embedding),top_k)
  retrived_texts = [chunks[i] for i in indices[0]]
  context=" ".join(retrived_texts)

  prompt = f"context: {context} \n\nQuestion {query} \nAnswer:"
  result = generation_pipeline(prompt,max_length=100)
  return result[0]['generated_text']



config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
#Here we will be getting users query and calling retrieve_and_generate function by  passing the user query
#by applying this function it will generate the answer after that we are printing it This is how RAG Works.
Question = "What is LLMs?"
Answer = retrieve_and_generate(Question)
print("Q:",Question)
print("A:",Answer)

Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Q: What is LLMs?
A: handle a wide range of tasks
